# Model Trainer
(As opposed to the controller part of the model)

## Loading stuff into the kernel
Imports and training data (don't need the encoder, train data is already converted)

In [1]:
# imports
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, MaxPooling2D,LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import save_model
import numpy as np
import matplotlib.pyplot as plt
import os
import joblib
import cv2
import random
import glob

In [2]:
data = joblib.load("images/train_data_rnn.z")

In [3]:
train_data = np.array([np.array(p[0])for p in data])
answers = np.array([np.array(p[1]) for p in data])
print(train_data[0])
print(answers[0])
print(np.shape(train_data[0]))
z_len = np.shape(train_data[0])[-1]
print(z_len)

[[ 0.0104093  -0.7574463   0.45002717 ... -1.2421685  -0.7610112
  -1.6004298 ]]
[[-0.87825227 -0.3997757  -1.2685652  ... -0.304825    0.14091796
  -0.38791984]]
(1, 2048)
2048


## Building the RNN

In [11]:
# Layers
input_to_rnn = Input(shape=(1,z_len))

x = LSTM(z_len, return_sequences=True)(input_to_rnn)
x = Dropout(0.2)(x)
x = Dense(z_len)(x)
x = Dropout(0.2)(x)

output = Dense(2048, activation='sigmoid')(x)

In [12]:
rnn = Model(input_to_rnn, output, name='rnn')

In [13]:
rnn.summary()

Model: "rnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 2048)]         0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 2048)           33562624  
                                                                 
 dropout_2 (Dropout)         (None, 1, 2048)           0         
                                                                 
 dense_2 (Dense)             (None, 1, 2048)           4196352   
                                                                 
 dropout_3 (Dropout)         (None, 1, 2048)           0         
                                                                 
 dense_3 (Dense)             (None, 1, 2048)           4196352   
                                                                 
Total params: 41,955,328
Trainable params: 41,955,328
Non-train

In [7]:
rnn.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
print(np.shape(answers[0]))

(1, 2048)


In [9]:
# after testing with z length = 2048, 10 epochs seems like the best. val_loss goes from 0.90 to 0.82, but goes back to
#   back to 0.83 for overfitting. If more data added, could potentially do more epochs
#   also, doesn't take that long to run due to small dimensionality of inputs and outputs
rnn.fit(train_data, answers, epochs=10, verbose = 1, batch_size = 32, validation_split=0.2)

Train on 602 samples, validate on 151 samples
Epoch 1/10
602/602 [==============================] - 11s 18ms/sample - loss: 0.9766 - val_loss: 0.8943
Epoch 2/10


C:\Users\mattp\anaconda3\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


602/602 [==============================] - 4s 7ms/sample - loss: 0.9074 - val_loss: 0.8582
Epoch 3/10
602/602 [==============================] - 4s 7ms/sample - loss: 0.8828 - val_loss: 0.8284
Epoch 4/10
602/602 [==============================] - 4s 7ms/sample - loss: 0.8661 - val_loss: 0.8260
Epoch 5/10
602/602 [==============================] - 4s 7ms/sample - loss: 0.8558 - val_loss: 0.8212
Epoch 6/10
602/602 [==============================] - 4s 7ms/sample - loss: 0.8484 - val_loss: 0.8195
Epoch 7/10
602/602 [==============================] - 4s 7ms/sample - loss: 0.8433 - val_loss: 0.8195
Epoch 8/10
602/602 [==============================] - 4s 7ms/sample - loss: 0.8384 - val_loss: 0.8201
Epoch 9/10
602/602 [==============================] - 4s 7ms/sample - loss: 0.8332 - val_loss: 0.8174
Epoch 10/10
602/602 [==============================] - 4s 7ms/sample - loss: 0.8267 - val_loss: 0.8218


In [10]:
rnn.save_weights('models/rnn.h5')